### [KOR]
- <span style = 'font-size:1.2em;line-height:1.5em'><b>3. </b>아래 데이터는 konlpy에서 제공하는 naver영화 평점 데이터이다. 아래 데이터를 보고, document column에 있는 text들을 전처리해보세요.</span>

    - <span style = 'font-size:1.1em;line-height:1.5em'> 띄어쓰기, 오타 교정, 형태소 분석기를 활용한 tokenizing, normalizing, word_dictionary 생성(빈도수가 상위 500개), Integer encoding, DTM 생성, TF-IDF생성</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'> DTM, TF-IDF의 각 row vector는 500차원의 벡터이다.</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'> 만약 Memory Error가 발생한다면 차원수를 더 줄여도 된다.</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'> DTM이나 TF-IDF를 생성할 때, OOV문제는 무시하여 벡터에 반영하지 않아도 된다.</span>

- 오타 교정 불가 (hanspell 이용 불가, selenium 시간 이슈로 불가)
- 띄어쓰기 불가 (Kospacing 속도 이슈로 불가)

In [36]:
from selenium import webdriver
from pykospacing import Spacing
import pickle
import pandas as pd
import numpy as np
import re
import urllib.request
from konlpy.tag import Okt
from tqdm import tqdm
from konlpy.tag import Okt
import requests
import json
from hanspell import spell_checker
import time
from selenium.webdriver.common.by import By
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

('ratings_test.txt', <http.client.HTTPMessage at 0x1f1aef47a60>)

In [3]:
train_data = pd.read_table('ratings_train.txt')
test_data = pd.read_table('ratings_test.txt')

In [4]:
train_data.drop_duplicates(subset = ['document'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
train_data['document'] = train_data['document'].str.replace('^ +', "") # 공백은 empty 값으로 변경
train_data['document'].replace('', np.nan, inplace=True) # document column이 비어있다면(공백), Null 값으로 변경
train_data = train_data.dropna(how='any') # Null 값 제거
print('전처리 후 학습용 샘플의 개수 :',len(train_data))

C:\Users\user\AppData\Local\Temp\ipykernel_16376\1209056594.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
C:\Users\user\AppData\Local\Temp\ipykernel_16376\1209056594.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  train_data['document'] = train_data['document'].str.replace('^ +', "") # 공백은 empty 값으로 변경


전처리 후 학습용 샘플의 개수 : 145393


In [5]:
documents = train_data['document'].tolist()
documents

['아 더빙 진짜 짜증나네요 목소리',
 '흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나',
 '너무재밓었다그래서보는것을추천한다',
 '교도소 이야기구먼 솔직히 재미는 없다평점 조정',
 '사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다',
 '막 걸음마 뗀 세부터 초등학교 학년생인 살용영화ㅋㅋㅋ별반개도 아까움',
 '원작의 긴장감을 제대로 살려내지못했다',
 '별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지정말 발로해도 그것보단 낫겟다 납치감금만반복반복이드라마는 가족도없다 연기못하는사람만모엿네',
 '액션이 없는데도 재미 있는 몇안되는 영화',
 '왜케 평점이 낮은건데 꽤 볼만한데 헐리우드식 화려함에만 너무 길들여져 있나',
 '걍인피니트가짱이다진짜짱이다',
 '볼때마다 눈물나서 죽겠다년대의 향수자극허진호는 감성절제멜로의 달인이다',
 '울면서 손들고 횡단보도 건널때 뛰쳐나올뻔 이범수 연기 드럽게못해',
 '담백하고 깔끔해서 좋다 신문기사로만 보다 보면 자꾸 잊어버린다 그들도 사람이었다는 것을',
 '취향은 존중한다지만 진짜 내생에 극장에서 본 영화중 가장 노잼 노감동임 스토리도 어거지고 감동도 어거지',
 'ㄱ냥 매번 긴장되고 재밋음ㅠㅠ',
 '참 사람들 웃긴게 바스코가 이기면 락스코라고 까고바비가 이기면 아이돌이라고 깐다그냥 까고싶어서 안달난것처럼 보인다',
 '굿바이 레닌 표절인것은 이해하는데 왜 뒤로 갈수록 재미없어지냐',
 '이건 정말 깨알 캐스팅과 질퍽하지않은 산뜻한 내용구성이 잘 버무러진 깨알일드',
 '약탈자를 위한 변명 이라 저놈들은 착한놈들 절대 아닌걸요',
 '나름 심오한 뜻도 있는 듯 그냥 학생이 선생과 놀아나는 영화는 절대 아님',
 '보면서 웃지 않는 건 불가능하다',
 '재미없다 지루하고 같은 음식 영화인데도 바베트의 만찬하고 넘 차이남바베트의 만찬은 이야기도 있고 음식 보는재미도 있는데  이건 볼게없다 음식도 별로 안나오고 핀란드 풍경이라도 구경할랫는데 그것

In [ ]:
'''
chrome_options = webdriver.ChromeOptions()
wd = webdriver.Chrome('C:\\Users\\user\\Downloads\\chromedriver.exe')

def spell_checking(sentence, text_list):
    time.sleep(0.5)
    wd.get('http://speller.cs.pusan.ac.kr/')
    try:
        wd.find_element(By.XPATH, '//*[@id="text1"]').send_keys(sentence)
        wd.find_element(By.XPATH, '//*[@id="btnCheck"]').click()
        time.sleep(0.5)
        entity_num = 0
        while True:
            try:
                wd.find_element(By.XPATH, '//*[@id="tdReplaceWord_'+str(entity_num)+'"]/ul/li/a').click()
                entity_num += 1
            except:
                break
        texts = wd.find_element(By.XPATH, '//*[@id="tdCorrection1stBox"]').text
        text_list.append(texts)
    except:
         text_list.append(sentence)
         
train_spell_list = []
for i in documents:
    spell_checking(i, train_spell_list)
'''

In [ ]:
'''
%%time
spacing = Spacing()
spaced_list = []
for i in range(len(documents)):
    spaced_list.append(spacing(documents[i]))
'''

In [6]:
okt = Okt()

In [7]:
lemmantized_list =[]
for i in range(len(documents)):
    lemmantized_list.append(okt.morphs(documents[i], stem=True)) # lemmantization
lemmantized_list

[['아', '더빙', '진짜', '짜증나다', '목소리'],
 ['흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '조차', '가볍다', '않다'],
 ['너', '무재', '밓었', '다그', '래서', '보다', '추천', '한', '다'],
 ['교도소', '이야기', '구먼', '솔직하다', '재미', '는', '없다', '평점', '조정'],
 ['사이',
  '몬페',
  '그',
  '의',
  '익살스럽다',
  '연기',
  '가',
  '돋보이다',
  '영화',
  '스파이더맨',
  '에서',
  '늙다',
  '보이다',
  '하다',
  '커스틴',
  '던스트',
  '가',
  '너무나도',
  '이쁘다',
  '보이다'],
 ['막',
  '걸음',
  '마',
  '떼다',
  '세',
  '부터',
  '초등학교',
  '학년',
  '생인',
  '살다',
  '영화',
  'ㅋㅋㅋ',
  '별',
  '반개',
  '도',
  '아깝다',
  '움'],
 ['원작', '의', '긴장감', '을', '제대로', '살리다', '하다'],
 ['별',
  '반개',
  '도',
  '아깝다',
  '욕',
  '나오다',
  '이응경',
  '길용우',
  '연',
  '기',
  '생활',
  '이',
  '몇',
  '년',
  '인지',
  '정말',
  '발',
  '로',
  '해도',
  '그것',
  '보단',
  '낫다',
  '납치',
  '감금',
  '만',
  '반복',
  '반복',
  '이',
  '드라마',
  '는',
  '가족',
  '도',
  '없다',
  '연기',
  '못',
  '하다',
  '사람',
  '만',
  '모',
  '엿',
  '네'],
 ['액션', '이', '없다', '재미', '있다', '몇', '안되다', '영화'],
 ['왜케',
  '평점',
  '이',
  '낮다',
  '꽤',
  '볼',
  '만',
  '한',
  '데'

In [12]:
with open('stopwords.txt', 'r', encoding='utf-8') as f:
    stopwords = f.readlines()
stopwords=[x.replace('\n','') for x in stopwords]

In [13]:
removal_stopwords = []
for i in range(len(lemmantized_list)):
    for j in lemmantized_list[i]:
        if j not in stopwords:
            removal_stopwords.append(j)

In [14]:
removal_stopwords

['더빙',
 '진짜',
 '짜증나다',
 '목소리',
 '포스터',
 '보고',
 '초딩',
 '영화',
 '줄',
 '오버',
 '연기',
 '가볍다',
 '않다',
 '무재',
 '밓었',
 '다그',
 '래서',
 '보다',
 '추천',
 '한',
 '다',
 '교도소',
 '이야기',
 '구먼',
 '솔직하다',
 '재미',
 '는',
 '없다',
 '평점',
 '조정',
 '사이',
 '몬페',
 '익살스럽다',
 '연기',
 '돋보이다',
 '영화',
 '스파이더맨',
 '늙다',
 '보이다',
 '하다',
 '커스틴',
 '던스트',
 '너무나도',
 '이쁘다',
 '보이다',
 '막',
 '걸음',
 '마',
 '떼다',
 '세',
 '초등학교',
 '학년',
 '생인',
 '살다',
 '영화',
 '별',
 '반개',
 '도',
 '아깝다',
 '움',
 '원작',
 '긴장감',
 '제대로',
 '살리다',
 '하다',
 '별',
 '반개',
 '도',
 '아깝다',
 '욕',
 '나오다',
 '이응경',
 '길용우',
 '연',
 '기',
 '생활',
 '인지',
 '정말',
 '발',
 '해도',
 '그것',
 '보단',
 '낫다',
 '납치',
 '감금',
 '만',
 '반복',
 '반복',
 '드라마',
 '는',
 '가족',
 '도',
 '없다',
 '연기',
 '못',
 '하다',
 '사람',
 '만',
 '엿',
 '액션',
 '없다',
 '재미',
 '안되다',
 '영화',
 '왜케',
 '평점',
 '낮다',
 '꽤',
 '볼',
 '만',
 '한',
 '데',
 '헐리우드',
 '식',
 '화려하다',
 '너무',
 '길들이다',
 '걍',
 '인피니트',
 '짱',
 '이다',
 '진짜',
 '짱',
 '이다',
 '볼때',
 '마다',
 '눈물나다',
 '죽다',
 '년대',
 '향수',
 '극',
 '허진호',
 '는',
 '감성',
 '절제',
 '멜로',
 '달인',
 '이다',
 '울면',
 '서',
 '손',
 

In [17]:
cnt = Counter(removal_stopwords)
print(cnt)

Counter({'영화': 50367, '보다': 41072, '하다': 40807, '도': 21079, '는': 16930, '은': 16011, '없다': 15572, '이다': 13196, '좋다': 12001, '너무': 11124, '다': 10105, '정말': 9720, '한': 9625, '되다': 8852, '재밌다': 8802, '적': 8747, '만': 8659, '진짜': 8312, '아니다': 7753, '않다': 7722, '점': 7547, '만들다': 6507, '나오다': 6397, '연기': 6326, '평점': 6266, '내': 5919, '최고': 5785, '안': 5495, '인': 5469, '스토리': 5332, '생각': 5313, '못': 5295, '드라마': 5044, '게': 4928, '감동': 4844, '사람': 4836, '보고': 4658, '이렇다': 4590, '말': 4527, '고': 4501, '아깝다': 4419, '더': 4352, '배우': 4286, '감독': 4114, '거': 4081, '그냥': 4062, '재미있다': 4038, '요': 4029, '재미': 3879, '내용': 3813, '뭐': 3778, '중': 3728, '주다': 3724, '자다': 3626, '하고': 3563, '지루하다': 3508, '재미없다': 3480, '쓰레기': 3434, '수': 3433, '모르다': 3209, '가다': 3209, '들다': 3197, '그렇다': 3095, '싶다': 3054, '지': 3027, '작품': 2978, '사랑': 2941, '알다': 2940, '다시': 2813, '마지막': 2756, '볼': 2738, '잘': 2714, '이건': 2592, '정도': 2585, '완전': 2526, '오다': 2513, '많다': 2462, '처음': 2453, '장면': 2425, '액션': 2394, '주인공': 2375, '이렇게': 2327, 

In [18]:
vocab = cnt.most_common(500)
print(vocab)

[('영화', 50367), ('보다', 41072), ('하다', 40807), ('도', 21079), ('는', 16930), ('은', 16011), ('없다', 15572), ('이다', 13196), ('좋다', 12001), ('너무', 11124), ('다', 10105), ('정말', 9720), ('한', 9625), ('되다', 8852), ('재밌다', 8802), ('적', 8747), ('만', 8659), ('진짜', 8312), ('아니다', 7753), ('않다', 7722), ('점', 7547), ('만들다', 6507), ('나오다', 6397), ('연기', 6326), ('평점', 6266), ('내', 5919), ('최고', 5785), ('안', 5495), ('인', 5469), ('스토리', 5332), ('생각', 5313), ('못', 5295), ('드라마', 5044), ('게', 4928), ('감동', 4844), ('사람', 4836), ('보고', 4658), ('이렇다', 4590), ('말', 4527), ('고', 4501), ('아깝다', 4419), ('더', 4352), ('배우', 4286), ('감독', 4114), ('거', 4081), ('그냥', 4062), ('재미있다', 4038), ('요', 4029), ('재미', 3879), ('내용', 3813), ('뭐', 3778), ('중', 3728), ('주다', 3724), ('자다', 3626), ('하고', 3563), ('지루하다', 3508), ('재미없다', 3480), ('쓰레기', 3434), ('수', 3433), ('모르다', 3209), ('가다', 3209), ('들다', 3197), ('그렇다', 3095), ('싶다', 3054), ('지', 3027), ('작품', 2978), ('사랑', 2941), ('알다', 2940), ('다시', 2813), ('마지막', 2756), ('볼', 2738),

In [19]:
word_dict = dict()
for i, (word, freq) in enumerate(vocab):
    word_dict[word] = i
print(word_dict)

{'영화': 0, '보다': 1, '하다': 2, '도': 3, '는': 4, '은': 5, '없다': 6, '이다': 7, '좋다': 8, '너무': 9, '다': 10, '정말': 11, '한': 12, '되다': 13, '재밌다': 14, '적': 15, '만': 16, '진짜': 17, '아니다': 18, '않다': 19, '점': 20, '만들다': 21, '나오다': 22, '연기': 23, '평점': 24, '내': 25, '최고': 26, '안': 27, '인': 28, '스토리': 29, '생각': 30, '못': 31, '드라마': 32, '게': 33, '감동': 34, '사람': 35, '보고': 36, '이렇다': 37, '말': 38, '고': 39, '아깝다': 40, '더': 41, '배우': 42, '감독': 43, '거': 44, '그냥': 45, '재미있다': 46, '요': 47, '재미': 48, '내용': 49, '뭐': 50, '중': 51, '주다': 52, '자다': 53, '하고': 54, '지루하다': 55, '재미없다': 56, '쓰레기': 57, '수': 58, '모르다': 59, '가다': 60, '들다': 61, '그렇다': 62, '싶다': 63, '지': 64, '작품': 65, '사랑': 66, '알다': 67, '다시': 68, '마지막': 69, '볼': 70, '잘': 71, '이건': 72, '정도': 73, '완전': 74, '오다': 75, '많다': 76, '처음': 77, '장면': 78, '액션': 79, '주인공': 80, '이렇게': 81, '안되다': 82, '걸': 83, '차다': 84, '나다': 85, '개': 86, '최악': 87, '돈': 88, '이야기': 89, '지금': 90, '넘다': 91, '느낌': 92, '임': 93, '연출': 94, 'ㅡㅡ': 95, '라': 96, '인데': 97, '듯': 98, '끝': 99, '명작': 100, '좋아하다':

In [42]:
vector = CountVectorizer(vocabulary = word_dict)

# 코퍼스로부터 각 단어의 빈도수를 기록
print('bag of words vector :', vector.fit_transform([documents[1234]]).toarray())

# 각 단어의 인덱스가 어떻게 부여되었는지를 출력
print('vocabulary :',vector.vocabulary_)

bag of words vector : [[0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 

In [32]:
vector = CountVectorizer(vocabulary=word_dict)
results = vector.fit_transform(documents).toarray()
results

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [2, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [39]:
vector = TfidfVectorizer(vocabulary=word_dict)
tfidf = vector.fit_transform(documents).toarray()
tfidf

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.68125303, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])